Detection Of Road Lane Line

In [1]:
pip install matplotlib numpy opencv-python moviepy


Imports:

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
import math

Apply frame masking and find region of interest:

In [3]:
def interested_region(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    return cv2.bitwise_and(img, mask)

Conversion of pixels to a line in Hough Transform space:

In [4]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_img, (x1, y1), (x2, y2), (255, 0, 0), 3)
    return line_img

Create two lines in each frame after Hough transform:

In [5]:
def lines_drawn(img, lines, color=[255, 0, 0], thickness=6):
    global cache
    global first_frame
    slope_l, slope_r = [],[]
    lane_l,lane_r = [],[]

    α = 0.2
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if slope > 0.4:
                slope_r.append(slope)
                lane_r.append(line)
            elif slope < -0.4:
                slope_l.append(slope)
                lane_l.append(line)
            img.shape[0] = min(y1, y2, img.shape[0])

    if ((len(lane_l) == 0) or (len(lane_r) == 0)):
        print('no lane detected')
        return 1

    slope_mean_l = np.mean(slope_l, axis=0)
    slope_mean_r = np.mean(slope_r, axis=0)
    mean_l = np.mean(np.array(lane_l), axis=0)
    mean_r = np.mean(np.array(lane_r), axis=0)

    if ((slope_mean_r == 0) or (slope_mean_l == 0)):
        print('dividing by zero')
        return 1

    x1_l = int((img.shape[0] - mean_l[0][1] - (slope_mean_l * mean_l[0][0])) / slope_mean_l)
    x2_l = int((img.shape[0] - mean_l[0][1] - (slope_mean_l * mean_l[0][0])) / slope_mean_l)
    x1_r = int((img.shape[0] - mean_r[0][1] - (slope_mean_r * mean_r[0][0])) / slope_mean_r)
    x2_r = int((img.shape[0] - mean_r[0][1] - (slope_mean_r * mean_r[0][0])) / slope_mean_r)

    if x1_l > x1_r:
        x1_l = int((x1_l + x1_r) / 2)
        x1_r = x1_l
        y1_l = int((slope_mean_l * x1_l) + mean_l[0][1] - (slope_mean_l * mean_l[0][0]))
        y1_r = int((slope_mean_r * x1_r) + mean_r[0][1] - (slope_mean_r * mean_r[0][0]))
        y2_l = int((slope_mean_l * x2_l) + mean_l[0][1] - (slope_mean_l * mean_l[0][0]))
        y2_r = int((slope_mean_r * x2_r) + mean_r[0][1] - (slope_mean_r * mean_r[0][0]))
    else:
        y1_l = img.shape[0]
        y2_l = img.shape[0]
        y1_r = img.shape[0]
        y2_r = img.shape[0]

    present_frame = np.array([x1_l, y1_l, x2_l, y2_l, x1_r, y1_r, x2_r, y2_r], dtype="float32")

    if first_frame == 1:
        next_frame = present_frame
        first_frame = 0
    else:
        prev_frame = cache
        next_frame = (1 - α) * prev_frame + α * present_frame

    cv2.line(img, (int(next_frame[0]), int(next_frame[1])), (int(next_frame[2]), int(next_frame[3])), color, thickness)
    cv2.line(img, (int(next_frame[4]), int(next_frame[5])), (int(next_frame[6]), int(next_frame[7])), color, thickness)

    cache = next_frame


Process each frame of video to detect lane:

In [6]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)


def process_image(image):

    global first_frame

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


    lower_yellow = np.array([20, 100, 100], dtype = "uint8")
    upper_yellow = np.array([30, 255, 255], dtype="uint8")

    mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(gray_image, 200, 255)
    mask_yw = cv2.bitwise_or(mask_white, mask_yellow)
    mask_yw_image = cv2.bitwise_and(gray_image, mask_yw)

    gauss_gray= cv2.GaussianBlur(mask_yw_image, (5, 5), 0)

    canny_edges=cv2.Canny(gauss_gray, 50, 150)

    imshape = image.shape
    lower_left = [imshape[1]/9,imshape[0]]
    lower_right = [imshape[1]-imshape[1]/9,imshape[0]]
    top_left = [imshape[1]/2-imshape[1]/8,imshape[0]/2+imshape[0]/10]
    top_right = [imshape[1]/2+imshape[1]/8,imshape[0]/2+imshape[0]/10]
    vertices = [np.array([lower_left,top_left,top_right,lower_right],dtype=np.int32)]
    roi_image = interested_region(canny_edges, vertices)

    theta = np.pi/180

    line_image = hough_lines(roi_image, 4, theta, 30, 100, 180)
    result = weighted_img(line_image, image, α=0.8, β=1., λ=0.)
    return result

In [7]:
import os

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: C:\Users\HP


In [8]:
from IPython.display import Video

In [9]:
input_filename = 'C:\\Users\\HP\\Downloads\\lane detection test.mp4'
input_file_path = os.path.join(current_directory, input_filename)
print("Full Path to Input Video File:", input_file_path)

Full Path to Input Video File: C:\Users\HP\Downloads\lane detection test.mp4


In [10]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Specify the directory path relative to the current working directory
directory_path = os.path.join(current_directory, 'path/to/your/output/video/')

# Create the directory
os.makedirs(directory_path, exist_ok=True)

print(f"Directory '{directory_path}' created successfully.")

Directory 'C:\Users\HP\path/to/your/output/video/' created successfully.


In [11]:
first_frame = 1
white_output = r'C:\Users\HP\path\to\your\output\video\output_video.mp4'

input_file_path = os.path.join(current_directory, 'C:\\Users\\HP\\Downloads\\lane detection test.mp4')
clip1 = VideoFileClip(input_file_path)
white_clip = clip1.fl_image(process_image)
white_clip.write_videofile(white_output, audio=False)

Moviepy - Building video C:\Users\HP\path\to\your\output\video\output_video.mp4.
Moviepy - Writing video C:\Users\HP\path\to\your\output\video\output_video.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\HP\path\to\your\output\video\output_video.mp4


In [ ]:
import tkinter as tk
from tkinter import *
import cv2
from PIL import Image, ImageTk
import os
import numpy as np

global last_frame1                                   
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
global last_frame2                                      
last_frame2 = np.zeros((480, 640, 3), dtype=np.uint8)
global cap1
global cap2

# Update the video file paths
cap1 = cv2.VideoCapture("C:\\Users\\HP\\Downloads\\lane detection test.mp4")
cap2 = cv2.VideoCapture(r'C:\Users\HP\path\to\your\output\video\output_video.mp4')

def show_vid():                                       
    if not cap1.isOpened():                             
        print("Can't open the camera1")
        return
    flag1, frame1 = cap1.read()
    if not flag1:
        print("Error reading frame from camera1")
        return
    frame1 = cv2.resize(frame1, (400, 500))
    global last_frame1
    last_frame1 = frame1.copy()
    pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB)     
    img = Image.fromarray(pic)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(10, show_vid)

def show_vid2():
    if not cap2.isOpened():                             
        print("Can't open the camera2")
        return
    flag2, frame2 = cap2.read()
    if not flag2:
        print("Error reading frame from camera2")
        return
    frame2 = cv2.resize(frame2, (400, 500))
    global last_frame2
    last_frame2 = frame2.copy()
    pic2 = cv2.cvtColor(last_frame2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(pic2)
    img2tk = ImageTk.PhotoImage(image=img2)
    lmain2.img2tk = img2tk
    lmain2.configure(image=img2tk)
    lmain2.after(10, show_vid2)

if __name__ == '__main__':
    root = tk.Tk()                                     
    lmain = tk.Label(master=root)
    lmain2 = tk.Label(master=root)

    lmain.pack(side=LEFT)
    lmain2.pack(side=RIGHT)
    root.title("Lane-line detection")            
    root.geometry("900x700+100+10") 
    exitbutton = Button(root, text='Quit', fg="red", command=root.destroy)
    exitbutton.pack(side=BOTTOM)
    show_vid()
    show_vid2()
    root.mainloop()

    # Release the video captures when the GUI is closed
    cap1.release()
    cap2.release()


Error reading frame from camera1
Error reading frame from camera2
